# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


In [2]:
# Import API key
from api_key import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
vacation_df = pd.read_csv("../output_data_file.csv")
vacation_df


,City,Latitude,Longitude,Temperature (max),Humidity,Cloudiness,Wind Speed,Country,Date
0,Torbay,47.6666,-52.7314,37.99,100,90,25.32,CA,1619841672
1,Bengkulu,-3.8004,102.2655,85.30,65,80,4.85,ID,1619841604
2,Kieta,-6.2167,155.6333,82.98,77,35,2.89,PG,1619841758
3,Rikitea,-23.1203,-134.9692,78.96,79,79,20.36,PF,1619841627
4,Tateyama,34.9833,139.8667,68.00,52,75,17.27,JP,1619841727
...,...,...,...,...,...,...,...,...,...
572,Iquique,-20.2208,-70.1431,62.60,67,0,1.86,CL,1619841746
573,Sur,22.5667,59.5289,86.25,49,61,3.83,OM,1619841705
574,Tomé,-36.6167,-72.9500,55.40,100,90,14.97,CL,1619841999
575,Plaridel,14.8872,120.8572,93.20,52,20,5.75,PH,1619841999


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
gmaps.configure(api_key=g_key)

In [12]:
vacation_df["Humidity"]

0      100
1       65
2       77
3       79
4       52
      ... 
572     67
573     49
574    100
575     52
576     91
Name: Humidity, Length: 577, dtype: int64

In [16]:
locations = vacation_df[["Latitude", "Longitude"]]
humidity = vacation_df["Humidity"].astype(float)

fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [43]:
# * Narrow down the DataFrame to find your ideal weather condition. For example:
#   * A max temperature lower than 80 degrees but higher than 70.
#   * Wind speed less than 10 mph.
#   * Zero cloudiness.

ideal_df = vacation_df.loc[vacation_df['Temperature (max)'] > 70]
more_ideal_df = ideal_df.loc[ideal_df['Temperature (max)'] < 80]
almost_ideal_df = more_ideal_df.loc[more_ideal_df['Wind Speed'] < 10]
most_ideal_df = almost_ideal_df.loc[almost_ideal_df['Cloudiness'] == 0]
most_ideal_df

,City,Latitude,Longitude,Temperature (max),Humidity,Cloudiness,Wind Speed,Country,Date
46,Hobart,-42.8794,147.3294,73.40,35,0,6.91,AU,1619841407
63,Cabo San Lucas,22.8909,-109.9124,73.00,70,0,1.01,MX,1619841551
174,Santa Rosalía,27.3167,-112.2833,74.17,32,0,5.70,MX,1619841900
200,Touros,-5.1989,-35.4608,73.40,94,0,4.61,BR,1619841907
217,Mossamedes,-15.1961,12.1522,75.13,70,0,5.50,AO,1619841690
321,Korla,41.7597,86.1469,79.32,11,0,7.00,CN,1619841677
336,Luxor,25.6989,32.6421,78.80,14,0,2.30,EG,1619841938
350,Kiryat Gat,31.6100,34.7642,72.00,59,0,6.91,IL,1619841647
355,Tanout,14.9709,8.8879,78.60,7,0,8.84,NE,1619841943
360,Zaoyang,32.1272,112.7542,76.01,37,0,3.89,CN,1619841944


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
